In [ ]:
using SBMLToolkit, ModelingToolkit, DifferentialEquations, StochasticDiffEq
using Plots
using DataFrames
using CSV
using Random
using Distributions
using SBML
using SymbolicUtils
using StaticArrays
using Catalyst
using AdvancedMH
using MCMCChains
using MCMCChainsStorage
using StatsPlots
using ArviZ
using HDF5

# Lorenzos packages
using Particles
using ParticlesDE
using StaticDistributions

include("utilities.jl");

In [ ]:
# calculate optimal sizes in pixels for latex documents textwidth, to create nice plots
dpi = 100 # use the Plots.jl default
width_pts = 455.244
inches_per_points = 1.0/72.27
width_inches = width_pts*inches_per_points
width_px = width_inches*dpi

## Introduction
In this notebook we covnert the SBML model to an ODE model, a jump process and an SDE model and investigate it visually. This notebook was used to create the figures used in Chapter 2.4 of the thesis. 
Afterwards, we focus on the SDE representation and introduce the observation function and create synthetic data. 

### Model import

In [ ]:
model_name = "covid_ethiopia_seir_variant_model_transformed_real_updated2"
petab_folder = "./petab_models/petab_virus_variant_model"
sbml_file = string(petab_folder, "/", model_name, ".sbml")
SBMLToolkit.checksupport_file(sbml_file)

In [34]:
# set the timespan
tspan = (0., 1000. )

# import the model from SBML
model = readSBML(sbml_file, doc -> begin
    set_level_and_version(3, 2)(doc)
    convert_simplify_math(doc)
end)

# create the reaction system
rs_sde = ReactionSystem(model, constraints=nothing)

# create the SDE-problem form the reaction system
SDEsys = convert(SDESystem, rs_sde)
SDE_problem = SDEProblem(SDEsys, [], tspan, []);
u0 = SDE_problem.u0 # store initial value

# create the jump process from the reaction system
prob = DiscreteProblem(rs_sde, u0, tspan, nothing)
jump_problem = JumpProblem(rs_sde, prob, Direct())

# create ODE-problem from the reaction system
ODE_problem = ODEProblem(rs_sde, [], tspan, []);

#100 (generic function with 1 method)

### Callbacks
In order to inject the variant into the population at time 170, we create a callback, that can be passed to the solvers.

In [ ]:
const tstop = [170.] # time variant enters
function condition(u,t,integrator)
    t in tstop
end
function affect!(integrator)
    integrator.u[10] = 1. # enters with one infected individual
end
save_positions = (true,true)

cb = DiscreteCallback(condition, affect!, save_positions=save_positions)

### SDE

In [ ]:
SDE_solution = solve(SDE_problem, PosEM(), dt=1e-2, dense=true, force_dtmin=true, callback=cb, tstops=tstop);
sde_plt = Plots.plot(SDE_solution, legend=false, size=(width_px/3, width_px/3))

In [ ]:
sde_rec = Plots.plot(SDE_solution.t, SDE_solution[Symbol("Rec_wild(t)")], label="Rec_wild(t)", size=(width_px/3, width_px/3))
Plots.plot!(sde_rec, SDE_solution.t, SDE_solution[Symbol("Rec_var(t)")], label="Rec_var(t)")
Plots.plot!(sde_rec, SDE_solution.t, SDE_solution[Symbol("Rec_var_wild(t)")], label="Rec_wild_var(t)")
plot!(sde_rec, legend=false, ylim=(0,300))
xlabel!(sde_rec, "t")

In [ ]:
savefig(sde_plt, "./figures/eth_full_sde_solution.pdf")
savefig(sde_rec, "./figures/eth_rec_sde_solution.pdf")

### Jump process

In [ ]:
jump_sol = solve(jump_problem, SSAStepper(),  callback=cb, tstops=tstop);
jump_plt = Plots.plot(jump_sol, legend=false, size=(width_px/3,width_px/3))

In [ ]:
jump_rec = Plots.plot(jump_sol.t, jump_sol[Symbol("Rec_wild(t)")], label="Rec_wild(t)", size=(width_px/3, width_px/3))
Plots.plot!(jump_rec, jump_sol.t, jump_sol[Symbol("Rec_var(t)")], label="Rec_var(t)")
Plots.plot!(jump_rec, jump_sol.t, jump_sol[Symbol("Rec_var_wild(t)")], label="Rec_wild_var(t)")
Plots.plot!(jump_rec, xlabel="t", legend=:false, ylim=(0,300))

In [ ]:
savefig(jump_plt, "./figures/eth_full_jump_solution.pdf")
savefig(plt, "./figures/eth_rec_jump_solution.pdf")

### ODE

In [ ]:
ode_sol = solve(ODE_problem; dense=true, solve_kwargs...);
ode_plt = Plots.plot(ode_sol, legend=false, size=(width_px/3,width_px/3))

In [ ]:
ode_rec = Plots.plot(sol.t, sol[Symbol("Rec_wild(t)")], label="Rec_wild(t)", size=(width_px/3, width_px/3))
Plots.plot!(ode_rec, sol.t, sol[Symbol("Rec_var(t)")], label="Rec_var(t)" )
Plots.plot!(ode_rec, sol.t, sol[Symbol("Rec_var_wild(t)")], label="Rec_wild_var(t)")
plot!(legend=false, ylim=(0,300))
xlabel!("t")

In [ ]:
savefig(ode_plt, "./figures/eth_full_ode_solution.pdf")
savefig(ode_rec, "./figures/eth_rec_ode_solution.pdf")

## Observation process

In [ ]:
# real observation function from petab-files
nobs = 2

function prev_comb(x, p, t)
    nom = (x[3]+x[4]+x[6]+x[8]+x[9])
    nsum = x[1]+x[2]+x[3]+x[4]+x[5]+x[6]+x[7]+x[8]+x[9]+x[10]
    return nom/nsum
end

function infc_rel(x, p, t)
    nom = x[3]+x[8]+x[9]
    nsum = x[1]+x[2]+x[3]+x[4]+x[5]+x[6]+x[7]+x[8]+x[9]+x[10]
    return nom/nsum
end

function fobs(x, p, t)

        return SIndependent(
            truncated(Normal(prev_comb(x, p, t), 0.027361836386980545; check_args=false), 0.0, nothing),
            truncated(Normal(infc_rel(x, p, t), 0.048944430985639616; check_args=false), 0.0, nothing),
            )
end

Load the synthetically generated data.

In [ ]:
# sparse measurements
tobs = [262, 308, 304, 328, 343, 353, 242, 273, 304, 333, 363, 393]
tobs = sort(tobs)
tobs = unique(tobs);

y_load = CSV.read("./data/real_model_SparseSynth.csv", DataFrame)

real_data = Vector{Vector{Union{Missing, Float64}}}()

for i in range(1,length(tobs))
    prev_meas = y_load[!, "y1"][Int64(i)]
    infc_meas = y_load[!, "y2"][Int64(i)]
    append!(real_data, [Vector{Union{Missing, Float64}}([prev_meas, infc_meas])])
end

real_data = collect(SVector{2, Union{Missing, Float64}}, real_data)

### Additional solver specifications

In [ ]:
# callbacks for injection of variant
tevent = Float64(170.0)
jumpsize = Float64(5000.0)

make_event_cb(tevent, jumpsize) = DiscreteCallback((u,t,integrator) -> t == tevent, integrator -> integrator.u[10] += jumpsize)
cb = make_event_cb(tevent, jumpsize)

# solver settings
solve_alg = PosEM()
solve_kwargs = (dt=1e-2, callback=cb, tstops=[tevent])
nothing

## State Space model
Next, we need to define the state space model and pass it to a Bootstrap Filter to define the likelihood function, that gets maximized.

In [ ]:
# creating the SDEStateSpaceModel
ssm = SDEStateSpaceModel(SDE_problem, initial_state, fobs, nobs, tobs, solve_alg; solve_kwargs...);

# creating the likelihood function

function likelihood(nparticles)
    llh_ssm = LogLikelihood_NoGradient(ssm, real_data; nparticles=nparticles)
    llh = let llh_ssm = llh_ssm, ssm = ssm
        p -> begin
            tevent = Float64(p[4])
            jumpsize = Float64(1.0)
            ssm.solve_kwargs = (dt = 0.01, callback = make_event_cb(tevent, jumpsize), tstops = [tevent]) # NB must be of the same type (order included) as the template passed to SDEStateSpaceModel
            all(≥(0) ,p[begin:end-3]) || return -Inf64
            p_full = copy(ssm.sprob.p)
            p_full[1:4] = p[1:4]
            return llh_ssm(p_full)
        end
    end
    return llh
end